In [84]:
import os
import requests
import numpy as np
import pandas as pd
import json
import csv
import psycopg2 
from io import StringIO
import json
import sqlite3
 

from sqlalchemy import create_engine, text
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = None

#import write_to_db
#import importlib
#importlib.reload(write_to_db)

In [85]:
#print(os.getenv("DATABASE_URI"))

#connection = engine.connect()

def get_engine():
    # load_dotenv(dotenv_path="./config/dev.env")
    DATABASE_URI = os.getenv("DATABASE_URI")
    engine = create_engine(DATABASE_URI)
    return engine

In [86]:
def query_bbdt_db(query):
    engine = get_engine()
    connection = engine.connect()
    df = pd.read_sql(text(query), connection)  # sql alchemy
    connection.close()  # not sure if I need this
    engine.dispose()  # not sure if I need this
    df = df.loc[:, ~df.columns.duplicated()].copy()
    return df

In [87]:
query = "SELECT fdc_id, description, brand_name, serving_size, serving_size_unit, branded_food_category, data_type, calories, carbs, protein, fat, monosaturated_fat, polysaturated_fat, saturated_fat, cholesterol, calcium, fiber, iron, sodium from branded_foods_nutrients" 

#query = "select * from users"
return_df = query_bbdt_db(query)
return_df.head(2)



,fdc_id,description,brand_name,serving_size,serving_size_unit,branded_food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium
0,1105904,WESSON Vegetable Oil 1 GAL,None,15.0,ml,Oils Edible,branded_food,130.05,0.00,0.00,14.0,3.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0
1,1105905,SWANSON BROTH BEEF,None,240.0,ml,Herbs/Spices/Extracts,branded_food,9.60,1.01,1.99,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,830.4


In [88]:
branded_df = return_df.copy()
branded_df.rename({"branded_food_category": "food_category"}, inplace=True, axis=1)
branded_df['description'] = branded_df['description'].str.title()
print(branded_df.shape)
branded_df.head(2)

(1590666, 19)


,fdc_id,description,brand_name,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium
0,1105904,Wesson Vegetable Oil 1 Gal,None,15.0,ml,Oils Edible,branded_food,130.05,0.00,0.00,14.0,3.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0
1,1105905,Swanson Broth Beef,None,240.0,ml,Herbs/Spices/Extracts,branded_food,9.60,1.01,1.99,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,830.4


In [89]:
branded_df[branded_df['fdc_id']==1456040]

,fdc_id,description,brand_name,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium
796379,1456040,Quaker Old Fashioned Oats 42Oz,Quaker,40.0,g,Grains/Flour,branded_food,150.0,27.0,5.0,3.0,1.0,1.0,0.5,0.0,20.0,4.0,1.5,0.0


In [90]:
query = "SELECT fdc_id, description, serving_size, serving_size_unit, category_description, data_type, calories, carbs, protein, fat, monosaturated_fat, polysaturated_fat, saturated_fat, cholesterol, calcium, fiber, iron, sodium from legacy_foods_nutrients" 
#return_df = write_to_db.query_bbdt_db(text(query))
return_df = query_bbdt_db(query)

In [91]:
legacy_df = return_df.copy()
legacy_df.rename({"category_description": "food_category"}, inplace=True, axis=1)
legacy_df['description'] = legacy_df['description'].str.title()
legacy_df.head(2)

,fdc_id,description,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",100.0,g,Baked Products,sr_legacy_food,796.5,41.18,5.88,13.24,NaN,NaN,2.94,0.0,NaN,1.2,2.12,1059.0
1,167513,"Pillsbury, Cinnamon Rolls With Icing, Refriger...",100.0,g,Baked Products,sr_legacy_food,855.5,53.42,4.34,11.27,NaN,NaN,3.25,0.0,28.0,1.4,1.93,780.0


In [92]:
def clean_nutrition_df(df, recalc_calories=False):
    print('start shape', df.shape)
    fill_nan_cols = ["carbs","protein","fat","monosaturated_fat","polysaturated_fat","saturated_fat","cholesterol","calcium","fiber","iron","sodium"]
    df[fill_nan_cols] = df[fill_nan_cols].fillna(0)
    print('na calories before', df[(df['calories'].isna())].shape)
    df['cal_temp_calc'] = df['carbs']*4 + df['protein']*4 + df['fat']*9
    if recalc_calories:
        df['calories']  = df['cal_temp_calc']
    else:
        df['calories'] = np.where(df['calories'].isna(), df['cal_temp_calc'], df['calories'])
    print('na calories after', df[(df['calories'].isna())].shape)
    df.drop('cal_temp_calc', axis=1, inplace=True)
    
    df = df[(~df['serving_size'].isna())]
    df = df[(~df['serving_size_unit'].isna())]
    print('end shape', df.shape)
    print(df[(df['calories'] == 0)].shape)
    return df


test_df = branded_df.copy()
branded_cleaned_df = clean_nutrition_df(test_df, recalc_calories=True)

test_df = legacy_df.copy()
legacy_cleaned_df = clean_nutrition_df(test_df, recalc_calories=True).drop_duplicates()
print(legacy_cleaned_df.shape)

start shape (1590666, 19)
na calories before (30092, 19)
na calories after (0, 20)
end shape (1579210, 19)
(72921, 19)
start shape (14709, 18)
na calories before (0, 18)
na calories after (0, 19)
end shape (14709, 18)
(74, 18)
(7793, 18)


In [93]:
def assign_high_category(row):
    if row["high_protein"] == 1:
        return 1
    elif row['high_carb'] == 1:
        return 2
    elif row["high_fat"] == 1:
        return 3
    else:
        return 0

def label_high_macro_foods(df, threshold):
    df['carb_perc'] = (df['carbs'] * 4)/df['calories']
    df['protein_perc'] = (df['protein'] * 4)/df['calories']
    df['fat_perc'] = (df['fat'] * 9)/df['calories']
    df["high_carb"] =np.where(df["carb_perc"]>=threshold,1,0)
    df["high_protein"] =np.where(df["protein_perc"]>=threshold,1,0)
    df["high_fat"] =np.where(df["fat_perc"]>=threshold,1,0)
    df["high_category"] = df.apply(assign_high_category, axis=1)
    return df

legacy_with_high_df = label_high_macro_foods(legacy_cleaned_df, threshold=0.8)
legacy_with_high_df['high_category'].value_counts()


high_category
0    5649
2    1551
3     366
1     227
Name: count, dtype: int64

In [94]:
branded_with_high_df = label_high_macro_foods(branded_cleaned_df, threshold=0.8)
branded_with_high_df['high_category'].value_counts()

high_category
0    1034207
2     449219
3      72808
1      22976
Name: count, dtype: int64

In [95]:
# combine food categories in simple groups
print(legacy_with_high_df['food_category'].nunique())
print(branded_with_high_df['food_category'].nunique())

25
355


In [96]:
branded_categories_rename_dict = {"Popcorn, Peanuts, Seeds & Related Snacks": "Snacks", "Candy": "Snacks", "Cheese": "Dairy and Egg Products",
                                 "Ice Cream & Frozen Yogurt": "Dairy and Egg Products", "Cookies & Biscuits": "Snacks", "Chips, Pretzels & Snacks": "Snacks",
                                 "Breads & Buns": "Baked Products", "Pickles, Olives, Peppers & Relishes": "Spices and Herbs", "Fruit & Vegetable Juice, Nectars & Fruit Drinks": "Fruits and Fruit Juices",
                                 "Chocolate": "Snacks", "Cereal": "Breakfast Cereals", "Snack, Energy & Granola Bars": "Snacks", "Cakes, Cupcakes, Snack Cakes": "Snacks", "Yogurt": "Dairy and Egg Products",
                                 "Seasoning Mixes, Salts, Marinades & Tenderizers": "Spices and Herbs", "Soda": "Sweets", "Other Snacks": "Snacks", "Ketchup, Mustard, BBQ & Cheese Sauce": "Spices and Herbs",
                                 "Dips & Salsa": "Spices and Herbs", "Pasta by Shape & Type": "Cereal Grains and Pasta", "Salad Dressing & Mayonnaise": "Spices and Herbs", "Wholesome Snacks": "Snacks",
                                 "Water": "Beverages", "Frozen Dinners & Entrees": "Meals, Entrees, and Side Dishes", "Frozen Appetizers & Hors D'oeuvres": "Meals, Entrees, and Side Dishes",
                                 "Pepperoni, Salami & Cold Cuts": "Sausages and Luncheon Meats", "Canned Vegetables": "Vegetables and Vegetable Products", "Frozen Vegetables" :"Vegetables and Vegetable Products",
                                 "Milk": "Dairy and Egg Products", "Vegetable & Cooking Oils": "Fats and Oils", "Sausages, Hotdogs & Brats": "Sausages and Luncheon Meats", 
                                 "Other Drinks": "Beverages", "Baking Decorations & Dessert Toppings": "Sweets", "Crackers & Biscotti": "Snacks","Powdered Drinks": "Beverages",
                                 "Pizza": "Baked Products", "Prepared Pasta & Pizza Sauces": "Cereal Grains and Pasta", "Frozen Fish & Seafood": "Finfish and Shellfish Products",
                                 "Canned & Bottled Beans": "Legumes and Legume Products", "Canned Fruit": "Fruits and Fruit Juices", "Oriental, Mexican & Ethnic Sauces": "Spices and Herbs",
                                 "Cake, Cookie & Cupcake Mixes": "Sweets", "Tomatoes": "Fruits and Fruit Juices", "Jam, Jelly & Fruit Spreads": "Sweets", 
                                 "Other Soups": "Soups, Sauces, and Gravies", "Nut & Seed Butters": "Legumes and Legume Products", "Croissants, Sweet Rolls, Muffins & Other Pastries": "Sweets",
                                 "Canned Soup": "Soups", "Frozen Fruit & Fruit Juice Concentrates": "Fruits and Fruit Juices", "Other Deli": "Sausages and Luncheon Meats",
                                 "Iced & Bottle Tea": "Beverages", "Other Meats": "Sausages and Luncheon Meats", "Other Cooking Sauces": "Soups, Sauces, and Gravies",
                                 "Energy, Protein & Muscle Recovery Drinks": "Bererages", "Syrups & Molasses": "Soups, Sauces, and Gravies", "Granulated, Brown & Powdered Sugar": "Sweets",
                                 "Milk Additives": "Beverages", "Butter & Spread": "Fats and Oils", "Herbs & Spices":  "Spices and Herbs", "Rice": "Cereal Grains and Pasta"}

                                 

In [97]:
branded_with_high_df.replace({"food_category": branded_categories_rename_dict}, inplace=True)
print(branded_with_high_df['food_category'].nunique())
branded_with_high_df['food_category'] = np.where(branded_with_high_df['food_category'].isin(list(branded_categories_rename_dict.values())), branded_with_high_df['food_category'], "Other")
print(branded_with_high_df['food_category'].nunique())

312
19


In [151]:
# combine_df
combined_df = pd.concat([legacy_with_high_df, branded_with_high_df], ignore_index=True)
print(combined_df.shape)
print(combined_df['food_category'].nunique())
combined_df.head(1)

(1587003, 26)
28


,fdc_id,description,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",100.0,g,Baked Products,sr_legacy_food,307.4,41.18,5.88,13.24,0.0,0.0,2.94,0.0,0.0,1.2,2.12,1059.0,0.535849,0.076513,0.387638,0,0,0,0,NaN


In [152]:
food_category_key_df = combined_df[['food_category']].drop_duplicates().sort_values('food_category').reset_index()
food_category_key_df['id'] = food_category_key_df.index + 1
food_category_key_df.drop('index', inplace=True, axis=1)
food_category_key_df.head(1)

,food_category,id
0,American Indian/Alaska Native Foods,1


In [153]:
#food_category_key_df.to_csv('./data/foods_export_ios/food_category_key')

In [154]:
combined_df = combined_df.merge(food_category_key_df, on='food_category', how='left')
combined_df.rename({'id': 'food_category_id'}, inplace=True, axis=1)
print(combined_df.shape)
combined_df.head(1)

(1587003, 27)


,fdc_id,description,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name,food_category_id
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",100.0,g,Baked Products,sr_legacy_food,307.4,41.18,5.88,13.24,0.0,0.0,2.94,0.0,0.0,1.2,2.12,1059.0,0.535849,0.076513,0.387638,0,0,0,0,NaN,3


In [155]:
print(combined_df[['fdc_id']].drop_duplicates().shape)

combined_df = combined_df.groupby(['description', 'food_category_id'], as_index=False).first()

(1587003, 1)


In [156]:
combined_df.shape

(404686, 27)

In [157]:
combined_df[combined_df['description'] == 'Blueberries']

,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name
40432,Blueberries,13,1106142,140.0,g,Fruits and Fruit Juices,branded_food,80.87,17.0,0.99,0.99,0.0,0.0,0.0,0.0,0.0,4.06,0.0,0.0,0.840856,0.048967,0.110177,1,0,0,2,BIRDS EYE
40433,Blueberries,18,522456,140.0,g,Other,branded_food,79.96,19.0,0.99,0.00,0.0,0.0,0.0,0.0,0.0,5.04,0.0,0.0,0.950475,0.049525,0.000000,1,0,0,2,NORTH BAY PRODUCE
40434,Blueberries,23,1139630,40.0,g,Snacks,branded_food,140.50,34.0,0.00,0.50,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.967972,0.000000,0.032028,1,0,0,2,STONERIDGE ORCHARDS


In [158]:
egg_whole_df = combined_df[combined_df['fdc_id'] == 173424].copy()
new_desc = "Egg, Whole"
egg_whole_df['description'] = new_desc
new_fdc_id = 10
egg_whole_df['fdc_id'] = new_fdc_id
new_serving_size = 1
egg_whole_df['serving_size'] = new_serving_size
new_serving_size_unit = 'whole'
egg_whole_df['serving_size_unit'] = new_serving_size_unit

cols_adjust = ['calories', 'carbs', 'protein', 'fat', 'monosaturated_fat', 'polysaturated_fat', 'saturated_fat', 'cholesterol', 'calcium', 'fiber', 'iron', 'sodium']
adjust_factor = 0.5

for c in cols_adjust:
    egg_whole_df[c] = egg_whole_df[c]*adjust_factor
combined_df = pd.concat([combined_df, egg_whole_df], ignore_index=True)
print(combined_df.shape)
egg_whole_df

(404687, 27)


,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name
118964,"Egg, Whole",9,10,1,whole,Dairy and Egg Products,sr_legacy_food,75.145,0.56,6.29,5.305,2.04,0.705,1.635,186.5,25.0,0.0,0.595,62.0,0.029809,0.334819,0.635372,0,0,0,0,None


In [159]:
egg_white_whole_df = combined_df[combined_df['fdc_id'] == 172183].copy()
new_desc = "Egg White, Whole"
egg_white_whole_df['description'] = new_desc
new_fdc_id = 11
egg_white_whole_df['fdc_id'] = new_fdc_id
new_serving_size = 1
egg_white_whole_df['serving_size'] = new_serving_size
new_serving_size_unit = 'whole'
egg_white_whole_df['serving_size_unit'] = new_serving_size_unit

cols_adjust = ['calories', 'carbs', 'protein', 'fat', 'monosaturated_fat', 'polysaturated_fat', 'saturated_fat', 'cholesterol', 'calcium', 'fiber', 'iron', 'sodium']
adjust_factor = 0.3

for c in cols_adjust:
    egg_white_whole_df[c] = egg_white_whole_df[c]*adjust_factor
combined_df = pd.concat([combined_df, egg_white_whole_df], ignore_index=True)
print(combined_df.shape)
egg_white_whole_df

(404688, 27)


,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name
118961,"Egg White, Whole",9,11,1,whole,Dairy and Egg Products,sr_legacy_food,14.415,0.219,3.27,0.051,0.0,0.0,0.0,0.0,2.1,0.0,0.024,49.8,0.06077,0.907388,0.031842,0,1,0,1,None


In [160]:
whole_df = combined_df[combined_df['fdc_id'] == 362759].copy()
new_desc = "Banana, Whole"
whole_df['description'] = new_desc
new_fdc_id = 12
whole_df['fdc_id'] = new_fdc_id
new_serving_size = 1
whole_df['serving_size'] = new_serving_size
new_serving_size_unit = 'whole'
whole_df['serving_size_unit'] = new_serving_size_unit

cols_adjust = ['calories', 'carbs', 'protein', 'fat', 'monosaturated_fat', 'polysaturated_fat', 'saturated_fat', 'cholesterol', 'calcium', 'fiber', 'iron', 'sodium']
adjust_factor = 1.3

for c in cols_adjust:
    whole_df[c] = whole_df[c]*adjust_factor
combined_df = pd.concat([combined_df, whole_df], ignore_index=True)
print(combined_df.shape)
whole_df

(404689, 27)


,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name
26153,"Banana, Whole",23,12,1,whole,Snacks,branded_food,117.91,28.6,0.0,0.39,0.0,0.0,0.0,0.0,0.0,0.65,0.702,2.548,0.970232,0.0,0.029768,1,0,0,2,NUTTY & FRUITY


In [161]:
whole_df = combined_df[combined_df['fdc_id'] == 498595].copy()
new_desc = "Apple, Whole"
whole_df['description'] = new_desc
new_fdc_id = 13
whole_df['fdc_id'] = new_fdc_id
new_serving_size = 1
whole_df['serving_size'] = new_serving_size
new_serving_size_unit = 'whole'
whole_df['serving_size_unit'] = new_serving_size_unit

cols_adjust = ['calories', 'carbs', 'protein', 'fat', 'monosaturated_fat', 'polysaturated_fat', 'saturated_fat', 'cholesterol', 'calcium', 'fiber', 'iron', 'sodium']
adjust_factor = 1.2

for c in cols_adjust:
    whole_df[c] = whole_df[c]*adjust_factor
combined_df = pd.concat([combined_df, whole_df], ignore_index=True)
print(combined_df.shape)
whole_df

(404690, 27)


,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name
16455,"Apple, Whole",13,13,1,whole,Fruits and Fruit Juices,branded_food,100.8,25.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.24,1.0,0.0,0.0,1,0,0,2,BEST CHOICE


In [162]:
combined_df.drop_duplicates(inplace=True)
combined_df.shape

(404690, 27)

In [163]:
new_table_name = 'food_category_id'
#food_category_key_df.to_sql(new_table_name, engine, method=write_to_db.psql_insert_copy, if_exists='replace', index=True)

In [164]:
# add custom bbdt foods
bbdt_protein_fdc_ids = {
    "Beef, Steak": 171759,
    "Beef, Ground": 174032,
    "Chicken, Breast": 171140,
    "Egg White, Whole": 11,
    "Egg, Whole": 10,
    "Fish, Tuna": 171986,
    "Fish, Salmon": 175175,
    "Hamburger, Ground": 174035,
    "Pork, lean": 168249,
    "Tofu, firm": 2440185,
    "Turkey, Breast": 172876,
    "Turkey, Ground": 172847,
    "Yogurt, Greek": 330137,
}
bbdt_carb_fdc_ids = {
    "Pasta, Cooked": 172026,
    "White Potato, Cooked": 170033,
    "Brown Rice, Cooked": 169704,
    "Oatmeal, Dry": 171661,
    "Quinoa, Cooked": 168917,
    "Sweet Potato, Cooked": 168483,
    "White Rice, Cooked": 168930,
    "Grain Ezekiel Bread": 2448803,
    "Cream of Wheat Cereal": 2159903,
    "Banana, Whole": 12,
    "Blueberries": 522456,
    "Apple": 13,
    "Watermelon": 533221,
    
}

bbdt_fat_fdc_ids = {
    "Almonds": 170567,
    "Acocado": 171705,
    "Flax Seed, Ground": 169414,
    "Peanut Butter": 172471,
    "Almond Butter": 168603,
    "Walnuts": 170594,
    "Hummus": 174289,
    "Cashews": 170162,
    "Hemp Seed, Hulled": 170148,
    
}

bbdt_veg_fdc_ids = {
    "Asparagus, Cooked": 168390,
    "Broccoli, Cooked": 168510,
    "Brussel Sprouts, Cooked": 168513,
    "Cauliflower, Cooked": 168520,
    "Celery, Raw": 169988,
    "Cucumber, Raw": 168409,
    "Green Beans, Cooked": 169965,
    "Kale, Raw": 169355,
    "Lettuce, Raw": 169249,
    "Peas, Cooked": 170420,
    "Peppers, Cooked": 170515,
    "Spinach, Cooked": 170531,
}

In [165]:
def custom_bbdt_foods_high_check(row):
    if row['fdc_id'] in list(bbdt_protein_fdc_ids.values()):
        return 1
    elif row['fdc_id'] in list(bbdt_carb_fdc_ids.values()):
        return 2
    elif row['fdc_id'] in list(bbdt_fat_fdc_ids.values()):
        return 3
    else:
        return row["high_category"]

def custom_bbdt_foods(row):
    if row['fdc_id'] in list(bbdt_protein_fdc_ids.values()):
        return 1
    elif row['fdc_id'] in list(bbdt_carb_fdc_ids.values()):
        return 2
    elif row['fdc_id'] in list(bbdt_fat_fdc_ids.values()):
        return 3
    elif row['fdc_id'] in list(bbdt_veg_fdc_ids.values()):
        return 4
    else:
        return 0
    
def custom_bbdt_foods_protein_bool(row):
    if row['bbdt_foods'] > 0:
        if row['fdc_id'] in list(bbdt_protein_fdc_ids.values()):
            return 1
        else:
            return 0
    else:
        return row["high_protein"]
    
def custom_bbdt_foods_carb_bool(row):
    if row['bbdt_foods'] > 0:
        if row['fdc_id'] in list(bbdt_carb_fdc_ids.values()):
            return 1
        else:
            return 0
    else:
        return row["high_carb"]
    
def custom_bbdt_foods_fat_bool(row):
    if row['bbdt_foods'] > 0:
        if row['fdc_id'] in list(bbdt_fat_fdc_ids.values()):
            return 1
        else:
            return 0
    else:
        return row["high_fat"]


In [166]:
combined_df["high_category"] = combined_df.apply(custom_bbdt_foods_high_check, axis=1)
combined_df["bbdt_foods"] = combined_df.apply(custom_bbdt_foods, axis=1)


In [167]:
print(combined_df["high_protein"].value_counts())
print(combined_df["high_carb"].value_counts())
print(combined_df["high_fat"].value_counts())

high_protein
0    399478
1      5212
Name: count, dtype: int64
high_carb
0    295523
1    109167
Name: count, dtype: int64
high_fat
0    389043
1     15647
Name: count, dtype: int64


In [168]:
combined_df.head(1)

,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name,bbdt_foods
0,Almond Milk,18,1276906,240.0,ml,Other,branded_food,58.5,7.99,1.01,2.5,1.49,0.5,0.0,0.0,100.8,0.96,0.36,148.8,0.546325,0.06906,0.384615,0,0,0,0,H-E-B,0


In [169]:
combined_df["high_protein"] =combined_df.apply(custom_bbdt_foods_protein_bool, axis=1)
combined_df["high_carb"] =combined_df.apply(custom_bbdt_foods_carb_bool, axis=1)
combined_df["high_fat"] =combined_df.apply(custom_bbdt_foods_fat_bool, axis=1)

In [170]:
print(combined_df["high_protein"].value_counts())
print(combined_df["high_carb"].value_counts())
print(combined_df["high_fat"].value_counts())

high_protein
0    399474
1      5216
Name: count, dtype: int64
high_carb
0    295523
1    109167
Name: count, dtype: int64
high_fat
0    389035
1     15655
Name: count, dtype: int64


In [171]:
combined_df['high_category'].value_counts()

high_category
0    274650
2    109169
3     15655
1      5216
Name: count, dtype: int64

In [172]:
combined_df[combined_df['fdc_id'].isin([10, 11, 12, 13])]

,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name,bbdt_foods
404686,"Egg, Whole",9,10,1.0,whole,Dairy and Egg Products,sr_legacy_food,75.145,0.560,6.29,5.305,2.04,0.705,1.635,186.5,25.0,0.00,0.595,62.000,0.029809,0.334819,0.635372,0,1,0,1,None,1
404687,"Egg White, Whole",9,11,1.0,whole,Dairy and Egg Products,sr_legacy_food,14.415,0.219,3.27,0.051,0.00,0.000,0.000,0.0,2.1,0.00,0.024,49.800,0.060770,0.907388,0.031842,0,1,0,1,None,1
404688,"Banana, Whole",23,12,1.0,whole,Snacks,branded_food,117.910,28.600,0.00,0.390,0.00,0.000,0.000,0.0,0.0,0.65,0.702,2.548,0.970232,0.000000,0.029768,1,0,0,2,NUTTY & FRUITY,2
404689,"Apple, Whole",13,13,1.0,whole,Fruits and Fruit Juices,branded_food,100.800,25.200,0.00,0.000,0.00,0.000,0.000,0.0,0.0,0.00,0.000,12.240,1.000000,0.000000,0.000000,1,0,0,2,BEST CHOICE,2


In [201]:
custom_bbdt_foods_desc = []
for index, row in combined_df[combined_df['fdc_id'].isin([10, 11, 12, 13])].iterrows():
    custom_bbdt_foods_desc.append(row['description'])
custom_bbdt_foods_desc

for c in custom_bbdt_foods_desc:
    print(c, combined_df[combined_df['description']==c].shape)

Egg, Whole (1, 28)
Egg White, Whole (1, 28)
Banana, Whole (1, 28)
Apple, Whole (1, 28)


In [173]:
# output for iOS stored nutrition
new_table_name = 'foods_ios_prepped'
engine = get_engine()
#combined_df.to_sql(new_table_name, engine, method=write_to_db.psql_insert_copy, if_exists='replace', index=True)
combined_df.to_sql(new_table_name, engine, if_exists="replace", index=True)

690

In [63]:
#cols_for_export = ['protein_perc', 'fat_perc', 'carb_perc', 'food_category_id']
#combined_df.drop(cols_for_export, axis=1, inplace=True)
#combined_df.to_csv("./data/foods_export_ios/food_table.tsv", sep ='\t')

# Tweak Output Table

In [174]:
query = "SELECT * from foods_ios_prepped" 


foods_ios_df = query_bbdt_db(query)

print(foods_ios_df.shape)
print(foods_ios_df["bbdt_foods"].value_counts())
try:
    foods_ios_df.drop(['level_0'], inplace=True, axis=1)
except:
    print('cols dont exist')
foods_ios_df.head(1)

(404690, 29)
bbdt_foods
0    404647
4        12
1        11
2        11
3         9
Name: count, dtype: int64
cols dont exist


,index,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name,bbdt_foods
0,0,Almond Milk,18,1276906,240.0,ml,Other,branded_food,58.5,7.99,1.01,2.5,1.49,0.5,0.0,0.0,100.8,0.96,0.36,148.8,0.546325,0.06906,0.384615,0,0,0,0,H-E-B,0


In [175]:
foods_ios_df["high_category"] = foods_ios_df.apply(custom_bbdt_foods_high_check, axis=1)
foods_ios_df["bbdt_foods"] = foods_ios_df.apply(custom_bbdt_foods, axis=1)

In [176]:
foods_ios_df["bbdt_foods"].value_counts()

bbdt_foods
0    404647
4        12
1        11
2        11
3         9
Name: count, dtype: int64

In [177]:
all_bbdt_dict = {**bbdt_protein_fdc_ids, **bbdt_carb_fdc_ids, **bbdt_fat_fdc_ids, **bbdt_veg_fdc_ids}

In [178]:
all_bbdt_dict_inv = dict(zip(all_bbdt_dict .values(), all_bbdt_dict .keys()))

In [179]:
def custom_bbdt_foods_brand_override(row):
    if row['fdc_id'] in list(all_bbdt_dict.values()):
        return "Bodybuilding Diet Food"
    else:
        return row["brand_name"]

def custom_bbdt_foods_desc_override(row):
    if row['fdc_id'] in list(all_bbdt_dict.values()):
        return all_bbdt_dict_inv[row['fdc_id']]
    else:
        return row["description"]

In [180]:
foods_ios_df["brand_name"] = foods_ios_df.apply(custom_bbdt_foods_brand_override, axis=1)

In [181]:
foods_ios_df["description"] = foods_ios_df.apply(custom_bbdt_foods_desc_override, axis=1)

In [182]:
chars_to_replace = ['"', '{', '}', '[', ']', '/']
for c in chars_to_replace:
    foods_ios_df['description'] = foods_ios_df['description'].str.replace(c, '')
foods_ios_df['description'] = foods_ios_df['description'].apply(lambda x: x.strip())

In [183]:
foods_ios_df['description'] = foods_ios_df['description'].str.replace(',', ', ')

In [184]:
foods_ios_df['word_count'] = foods_ios_df['description'].str.split().map(len)

In [185]:
def serving_size_assign(row):
    if row['serving_size_unit'] == "whole":
        return 0.25
    elif row["serving_size"] <= 1:
        return 0.1
    elif row['serving_size'] <= 30:
        return 1.0
    else:
        return 10.0 

foods_ios_df["serving_size_increment"] = foods_ios_df.apply(serving_size_assign, axis=1)
foods_ios_df["serving_min"] = 0
foods_ios_df["serving_max"] = foods_ios_df["serving_size"] * 20

In [186]:
foods_ios_df[foods_ios_df['fdc_id'].isin([10, 11, 12, 13])]

,index,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name,bbdt_foods,word_count,serving_size_increment,serving_min,serving_max
404686,404686,"Egg, Whole",9,10,1.0,whole,Dairy and Egg Products,sr_legacy_food,75.145,0.560,6.29,5.305,2.04,0.705,1.635,186.5,25.0,0.00,0.595,62.000,0.029809,0.334819,0.635372,0,1,0,1,Bodybuilding Diet Food,1,2,0.25,0,20.0
404687,404687,"Egg White, Whole",9,11,1.0,whole,Dairy and Egg Products,sr_legacy_food,14.415,0.219,3.27,0.051,0.00,0.000,0.000,0.0,2.1,0.00,0.024,49.800,0.060770,0.907388,0.031842,0,1,0,1,Bodybuilding Diet Food,1,3,0.25,0,20.0
404688,404688,"Banana, Whole",23,12,1.0,whole,Snacks,branded_food,117.910,28.600,0.00,0.390,0.00,0.000,0.000,0.0,0.0,0.65,0.702,2.548,0.970232,0.000000,0.029768,1,0,0,2,Bodybuilding Diet Food,2,2,0.25,0,20.0
404689,404689,Apple,13,13,1.0,whole,Fruits and Fruit Juices,branded_food,100.800,25.200,0.00,0.000,0.00,0.000,0.000,0.0,0.0,0.00,0.000,12.240,1.000000,0.000000,0.000000,1,0,0,2,Bodybuilding Diet Food,2,1,0.25,0,20.0


In [187]:
#foods_ios_df.drop(['high_carb', 'high_protein', 'high_fat'], inplace=True, axis=1)

In [188]:
#foods_ios_df[foods_ios_df['description'].str.contains(',')].head(3)
foods_ios_df[foods_ios_df['brand_name'] == 'Bodybuilding Diet Food'][['fdc_id', 'description', 'high_category', 'high_carb', 'high_protein', 'high_fat']].tail(100)


,fdc_id,description,high_category,high_carb,high_protein,high_fat
21011,168390,"Asparagus, Cooked",0,0,0,0
22917,171705,Acocado,3,0,0,1
29762,169965,"Green Beans, Cooked",0,0,0,0
31705,174035,"Hamburger, Ground",1,0,1,0
31706,174032,"Beef, Ground",1,0,1,0
32193,171759,"Beef, Steak",1,0,1,0
40437,522456,Blueberries,2,1,0,0
46311,168510,"Broccoli, Cooked",0,0,0,0
47610,168513,"Brussel Sprouts, Cooked",0,0,0,0
60696,168520,"Cauliflower, Cooked",0,0,0,0


In [189]:
foods_ios_df[foods_ios_df['brand_name'] == 'Bodybuilding Diet Food'].head(1)

,index,description,food_category_id,fdc_id,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,iron,sodium,carb_perc,protein_perc,fat_perc,high_carb,high_protein,high_fat,high_category,brand_name,bbdt_foods,word_count,serving_size_increment,serving_min,serving_max
21011,21005,"Asparagus, Cooked",28,168390,100.0,g,Vegetables and Vegetable Products,sr_legacy_food,28.02,4.11,2.4,0.22,0.0,0.1,0.05,0.0,23.0,2.0,0.91,14.0,0.586724,0.342612,0.070664,0,0,0,0,Bodybuilding Diet Food,4,2,10.0,0,2000.0


In [190]:
'''
cols_for_export = ['protein_perc', 'fat_perc', 'carb_perc', 'food_category_id']
foods_ios_df.drop(cols_for_export, axis=1, inplace=True)
foods_ios_df.to_csv("./data/foods_export_ios/food_table.tsv", sep ='\t')
'''

'\ncols_for_export = [\'protein_perc\', \'fat_perc\', \'carb_perc\', \'food_category_id\']\nfoods_ios_df.drop(cols_for_export, axis=1, inplace=True)\nfoods_ios_df.to_csv("./data/foods_export_ios/food_table.tsv", sep =\'\t\')\n'

In [191]:
new_table_name = 'foods_ios'
engine = get_engine()
foods_ios_df.to_sql(new_table_name, engine, if_exists="replace", index=True)

109

In [192]:
# run this in pgAdmin
# ALTER TABLE foods_ios ADD PRIMARY KEY (index)

In [193]:
conn = sqlite3.connect(r'./data/foods_export_ios/food_db.db')
foods_ios_df.to_sql('food_table', conn, if_exists='replace', index=False)
conn.close()

# Create a cursor object
conn = sqlite3.connect(r'./data/foods_export_ios/food_db.db')
cur = conn.cursor()
# Fetch and display result
for row in cur.execute('SELECT * FROM food_table'):
    print(row)
    break
# Close connection to SQLite database

conn.close()


(0, 'Almond Milk', 18, 1276906, 240.0, 'ml', 'Other', 'branded_food', 58.5, 7.99, 1.01, 2.5, 1.49, 0.5, 0.0, 0.0, 100.8, 0.96, 0.36, 148.8, 0.5463247863247863, 0.06905982905982906, 0.38461538461538464, 0, 0, 0, 0, 'H-E-B', 0, 2, 10.0, 0, 4800.0)


In [47]:
'''

conn = sqlite3.connect(r'./data/foods_export_ios/food_db_from_nikesh.db')
# Create a cursor object
cur = conn.cursor()
# Fetch and display result
#for row in cur.execute('SELECT * FROM sqlite_master WHERE type="table";'):
for row in cur.execute('SELECT * FROM food_table'):
    print(row)
    break
# Close connection to SQLite database

conn.close()

'''

(0, 'Almond Milk', 1276906, 240.0, 'ml', 'Other', 'branded_food', 60.0, 7.99, 1.01, 2.5, 1.49, 0.5, 0.0, 0.0, 100.8, 0.96, 0.36, 148.8, 0, 0, 0, 0, 0)


# end

In [96]:
#legacy_cleaned_df[(legacy_cleaned_df['protein_perc'] > 0.8) & (legacy_cleaned_df['protein'] > 10)].sort_values(['description'])

#legacy_cleaned_df[(legacy_cleaned_df['carb_perc'] > 0.8) & (legacy_cleaned_df['carbs'] > 10) & (legacy_cleaned_df['food_category'] == "Cereal Grains and Pasta")].sort_values(['description'])

food_search_term = "Beef"
legacy_cleaned_df[(legacy_cleaned_df['carb_perc'] < 0.9)\
                  & (legacy_cleaned_df['carbs'] > 0)\
                  & (legacy_cleaned_df['description'].str.contains(food_search_term))].sort_values(['description'])

In [ ]:
branded_cleaned_df[(branded_cleaned_df['protein_perc'] > 0.8) & (legacy_cleaned_df['protein'] > 10)].sort_values(['description'])

#legacy_cleaned_df[(legacy_cleaned_df['carb_perc'] > 0.8) & (legacy_cleaned_df['carbs'] > 10) & (legacy_cleaned_df['food_category'] == "Cereal Grains and Pasta")].sort_values(['description'])

food_search_term = "Mustard"
#legacy_cleaned_df[(legacy_cleaned_df['carb_perc'] < 0.9)\
#                  & (legacy_cleaned_df['carbs'] > 0)\
#                  & (legacy_cleaned_df['description'].str.contains(food_search_term))].sort_values(['description'])

## BBDT Chosen Foods

In [97]:
bbdt_protein_fdc_ids = {
    "Beef": 171759,
    "Chicken Breast": 171140,
    "Egg White": 172183,
    "Egg Whole": 173424,
    "Fish": 171986,
    "Pork": 168249,
    "Turkey Breast": 172876,
    "Turkey Ground": 172847,
}
bbdt_carb_fdc_ids = {
    "Pasta": 172026,
    "White Potato": 170033,
    "Brown Rice": 169704,
    "Oatmeal": 174660,
    "Quinoa": 168917,
    "Sweet Potato": 168483,
    "White Rice": 168930,
}

bbdt_fat_fdc_ids = {
    "Almonds": 170567,
    "Acocado": 171705,
    "Flax": 169414,
    "Peanut Butter": 172471,
    "Walnuts": 170594,
}

bbdt_veg_fdc_ids = {
    "Asparagus": 168390,
    "Broccoli": 168510,
    "Brussel Sprouts": 168513,
    "Cauliflower": 168520,
    "Celery": 169988,
    "Cucumber": 168409,
    "Green Beans": 169965,
    "Kale": 169355,
    "Lettuce": 169249,
    "Peas": 170420,
    "Peppers": 170515,
    "Spinach": 170531,
}


In [43]:
'''
day_week_key_dict = {"fdc_id": 2341370, "description": "Chicken Breast", 'serving_size': 60.0, "serving_size_unit": "g",
                    "food_category": "Poultry, Chicken & Turkey", "data_type": "bbdt", "calories": 120, "carbs": 0,
                    "protein": 25, "fat": 3.0, "monosaturated_fat": 1.5, "polysaturated_fat": 0.5, "saturated_fat": 1.0, "cholesterol": 75,
                     "calcium": 10, "fiber": 0,	"fiber_soluble": 0, "iron": 0.45, "sodium": 279}
new_foods_df = pd.DataFrame(day_week_key_dict, index=[0])
new_foods_df
'''

,fdc_id,description,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,fiber_soluble,iron,sodium
0,2341370,Chicken Breast,60.0,g,"Poultry, Chicken & Turkey",bbdt,120,0,25,3.0,1.5,0.5,1.0,75,10,0,0,0.45,279


In [44]:
'''
updated_df = df.append(new_foods_df)
print(updated_df.shape)
updated_df.tail(2)
'''

(1590702, 19)


,fdc_id,description,serving_size,serving_size_unit,food_category,data_type,calories,carbs,protein,fat,monosaturated_fat,polysaturated_fat,saturated_fat,cholesterol,calcium,fiber,fiber_soluble,iron,sodium
1590700,2272773,"Strawberry Preserves, Strawberry",20.0,g,"Jam, Jelly & Fruit Spreads",branded_food,50.0,13.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
0,2341370,Chicken Breast,60.0,g,"Poultry, Chicken & Turkey",bbdt,120.0,0.0,25.0,3.0,1.5,0.5,1.0,75.0,10.0,0.0,0.0,0.45,279.0


start shape (1590701, 18)
na calories before (30127, 18)
na calories after (0, 19)
end shape (1579245, 18)
(228, 18)
(0, 18)
(0, 18)
(78507, 18)
(0, 18)
(0, 18)
(0, 18)


In [5]:
df['food_category'].value_counts()

Popcorn, Peanuts, Seeds & Related Snacks                                    76049
Candy                                                                       74330
Cheese                                                                      62067
Ice Cream & Frozen Yogurt                                                   49344
Cookies & Biscuits                                                          46532
Chips, Pretzels & Snacks                                                    41291
Breads & Buns                                                               37499
Pickles, Olives, Peppers & Relishes                                         36792
Fruit & Vegetable Juice, Nectars & Fruit Drinks                             34018
Chocolate                                                                   32883
Cereal                                                                      32312
Snack, Energy & Granola Bars                                                32212
Cakes, Cupcakes,

# BBDT Foods

In [58]:
def get_bbdt_foods():
    bbdt_protein_fdc_ids = {
        "Beef": 171759,
        "Chicken Breast": 171140,
        "Egg White": 172183,
        "Egg Whole": 173424,
        "Fish": 171986,
        "Pork": 168249,
        "Turkey Breast": 172876,
        "Turkey Ground": 172847,
    }
    bbdt_carbs_fdc_ids = {
        "Pasta": 172026,
        "White Potato": 170033,
        "Brown Rice": 169704,
        "Oatmeal": 174660,
        "Quinoa": 168917,
        "Sweet Potato": 168483,
        "White Rice": 168930,
    }

    bbdt_fat_fdc_ids = {
        "Almonds": 170567,
        "Acocado": 171705,
        "Flax": 169414,
        "Peanut Butter": 172471,
        "Walnuts": 170594,
    }

    bbdt_veg_fdc_ids = {
        "Asparagus": 168390,
        "Broccoli": 168510,
        "Brussel Sprouts": 168513,
        "Cauliflower": 168520,
        "Celery": 169988,
        "Cucumber": 168409,
        "Green Beans": 169965,
        "Kale": 169355,
        "Lettuce": 169249,
        "Peas": 170420,
        "Peppers": 170515,
        "Spinach": 170531,
    }
    bbdt_protein_df = pd.DataFrame.from_dict(
        bbdt_protein_fdc_ids, orient="index"
    ).reset_index()
    bbdt_protein_df.columns = ["description", "fdc_id"]
    bbdt_protein_df["bbdt_foods_key"] = 1

    bbdt_carbs_df = pd.DataFrame.from_dict(
        bbdt_carbs_fdc_ids, orient="index"
    ).reset_index()
    bbdt_carbs_df.columns = ["description", "fdc_id"]
    bbdt_carbs_df["bbdt_foods_key"] = 2

    bbdt_fat_df = pd.DataFrame.from_dict(bbdt_fat_fdc_ids, orient="index").reset_index()
    bbdt_fat_df.columns = ["description", "fdc_id"]
    bbdt_fat_df["bbdt_foods_key"] = 3

    bbdt_veg_df = pd.DataFrame.from_dict(bbdt_veg_fdc_ids, orient="index").reset_index()
    bbdt_veg_df.columns = ["description", "fdc_id"]
    bbdt_veg_df["bbdt_foods_key"] = 4

    bbdt_rec_foods_df = pd.concat(
        [bbdt_protein_df, bbdt_carbs_df, bbdt_fat_df, bbdt_veg_df], ignore_index=True
    )
    return bbdt_rec_foods_df

In [60]:

'''
df = get_bbdt_foods()
new_table_name = 'bbdt_foods'
df.to_sql(new_table_name, engine, method=write_to_db.psql_insert_copy, if_exists='replace', index=True)

write_to_db.set_primiary_key(connection, new_table_name, 'id')
write_to_db.set_primiary_key(connection, new_table_name, create_index=True, primary_key_col=None, drop_col='index')
'''

"\ndf = get_bbdt_foods()\nnew_table_name = 'bbdt_foods'\ndf.to_sql(new_table_name, engine, method=write_to_db.psql_insert_copy, if_exists='replace', index=True)\n\nwrite_to_db.set_primiary_key(connection, new_table_name, 'id')\nwrite_to_db.set_primiary_key(connection, new_table_name, create_index=True, primary_key_col=None, drop_col='index')\n"